# Введение или просто добавь воды

In [ ]:
def commitChanges():
    ! git add KnowYourMemesParser.ipynb
    ! git commit -m "added function for full meme page parsing"
    ! git push -u origin master

In [ ]:
def commitPictures():
    ! git add pictures/
    ! git commit -m "added pictures"
    ! git push -u origin master

In [ ]:
commitChanges()
#commitPictures()

# Отсюда начинается реальный парсинг :з

Машинное обучение, эконометрика, статистика и многие другие науки о данных занимаются поиском закономерностей. Каждый день доблестные аналитики-инквизиторы пытают природу разными методами и вытаскивают из неё сведенья о том, как именно устроен великий процесс порождения данных, породивший нашу вселенную. Испанские инквизиторы в своей повседневной деятельности пытали непосредственно физическое тело своей жертвы. Природа вездесуща и не имеет однозначного физического облика. Из-за этого профессия современного инквизитора имеет странную специфику. Пытки природы происходят через анализ данных, которые надо откуда-то брать. Обычно данные инквизиторам приносят мирные собиратели. Эта статейка призвана немного приоткрыть завесу тайны с того откуда данные берутся и как их можно немножечко пособирать. 

Нашим девизом будет знаменитая фраза капитана Джэка Воробья: "Бери всё и не отдавай ничего". Собирать для дальнейших исследований и изысканий мы будем мемы. Иногда для их сбора мы будем использовать бандитские методы. Тем не менее, при всём при этом, мы будем оставаться мирными собирателями данных, и ни в коей мере не будем становиться бандитами. Брать мемы мы будем из [мемохранилища.](http://knowyourmeme.com)

<img align="center" src="https://d.justpo.st/media/images/2014/04/e4eec44d91001b3ab02407d48fe92075.jpg
" height="300" width="300"> 

# 1. Вламываемся в мемохранилище

## 1.1. Что мы хотим получить

Итак, мы хотим распарсить [knowyourmeme.com](http://knowyourmeme.com) и получить кучу разных переменных: 

- **Name** – название мема,
- **Origin_year** – год создания мема,
- **Views** – просматриваемость мема,
- ** и другие** 

Более того, мы хотим сделать это без вот этого всего: 

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/2.png" height="600" width="600">

И сделаем. После скачки и чистки данных от мусора, можно будет заняться строительством моделей. Например, попытаться предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой зубодробительных определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая бродит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг! 


## 1.2.  Что такое HTML 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега есть свои потомки (дети) — те теги, которые вложены в него и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div> 
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    
<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/tree.png" height="500" width="500"> 

Можно работать с этим html как с тексом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.

- [CSS-селектор](https://ru.wikibooks.org/wiki/CSS/%D0%A1%D0%B5%D0%BB%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D1%8B) (это когда мы ищем элемент страницы по паре ключ, значение)
- [XPath](https://ru.wikipedia.org/wiki/XPath) (это когда мы прописываем путь по дереву вот так: /html/body/div[1]/div[3]/div/div[2]/div)
- Всякие разные библиотеки для всяких разных языков, например, BeautifulSoup для питона. Именно эту библиотеку мы и будем использовать. 

## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов. 

In [ ]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени

Также подгрузим небольшой сюрприз. 

In [ ]:
# this is a surprise tool that will help us later!
from fake_useragent import UserAgent

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/memes_main.png" height="500" width="500"> 

Отсюда мы и будем тащить ссылки на каждый из перечисленных мемов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [ ]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'

In [ ]:
response = requests.get(page_link)
response

А вот и первая проблема! Обращаемся к [главному источнику знаний](https://en.wikipedia.org/wiki/HTTP_403) и выясняем, что 403-я ошибка выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать. 

**Картинка с котиком 403** 

Попробуем выяснить, почему. Для этого посмотрим, как выглядел финальный запрос, отправленный нами на сервер, а в первую очередь - как выглядел наш User-Agent в глазах сервера.

In [ ]:
for key, value in response.request.headers.items():
    print(key+": "+value)

**Диме: надо отмониторите номер библиотеки!**

Похоже, мы недвусмысленно дали понять серверу, что мы сидим на питоне и используем библиотеку requests под версией 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. Для сравнения, можно посмотреть, как выглядят request-headers у здорового человека:

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/good_headers.png" height="800" width="800"> 

Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [ ]:
UserAgent().chrome

In [ ]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [ ]:
html = response.content

In [ ]:
html[:1000]

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп

<img align="center" src="https://www.crummy.com/software/BeautifulSoup/10.1.jpg" height="200" width="200"> 

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это искаверканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`. 

In [ ]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [ ]:
soup = BeautifulSoup(html,'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их! 

Например, можно бродить по вершинам, указывая путь из тегов.

In [ ]:
soup.html.head.title

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [ ]:
soup.html.head.title.text

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [ ]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

Однако, вопреки нашим ожиданиям, вытащенный объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и задать строгое соответствие:

In [ ]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка. 

In [ ]:
obj.attrs['href']

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения. 

In [ ]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`. На выход будет выдан список! Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [ ]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

Осталось очистить полученный список от мусора:

In [ ]:
meme_links = [link.attrs['href'] for link in meme_links]

In [ ]:
meme_links[:10]

In [ ]:
len(meme_links)

Готово, получили ровно 16 ссылок по числу мемов на одной странице поиска. 

Окей, то, что можно искать элемент по его адресу, конечно же, круто, но откуда взять этот адрес? Можно установить для своего браузера какую-нибудь утилиту, позволяющую вытаскивать со страницы нужные теги. Например, [selectorgadget.](http://selectorgadget.com/)

Тем не менее, этот путь не подходит для истинного самурая. Для последователей бусидо есть другой способ — искать теги для каждого нужного нам элемента вручную. Для этого нам придётся жать правой кнопкой мышки по окошечку браузера и тыкать кнопку **Inspect**. Ваше окно браузера будет после всех этих манипуляций выглядеть как-то вот так: 

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/memes_inspection.png" height="800" width="800"> 

Если тыкнуть кнопочку, которая находится в огромном красном кружочке, а после навести курсорчик на требующийся вам кусочек странички и тыкнуть на него, то справа, там, где находится гигантская стрелочка, выскочит кусок html-кода, в котором находится адрес выбранного вами объекта. Этот адрес можно смело копировать в код и наслаждаться своей брутальностью. 

Остался последний момент. Когда мы скачаем все мемы с текущей страницы, нам нужно будет каким-то образом забраться на соседнюю. На сайте это можно делать просто пролистывая страничку с мемами вниз.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отображаются на структуре хрефа. Мемы не являются исключением. Если мы хотим получить первую порцию из мемов, мы должны будем обратиться к сайту по ссылке 

                `http://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить поцию из вторых 16 мемов, нам придётся немного видоизменить ссылку, а именно заменить номер страницы на второй.


                `http://knowyourmeme.com/memes/all/page/2`
 
Таким незатейливым образом мы сможем пройтись по всем страницам и ограбить мемохранилище. 

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [ ]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
            
    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]
    
    return meme_links

Протестируем функцию и убедимся, что всё хорошо

In [ ]:
meme_links = getPageLinks(1)
meme_links[:2]

In [ ]:
meme_links = getPageLinks(2)
meme_links[:2]

Отлично, функция работает и теперь мы теоретически можем достать ссылки на все $17171$ мем, для чего нам придется пройтись по $\frac{17171}{16} \approx 1074$ страницам. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме. 

## 1.5 Финальная подготовка к грабежу

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов: 

1. Открываем страничку с мемом
2. Находим тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit 

Для закрепления информации в голове любознательного читателя, вытащим число просмотров мема.

А в качестве примера возьмем самый популярный на этом сайте мем - Doge, набравший более 12 миллионов просмотров по состоянию на 18 ноября 2017 года. 

Сама страница, с которой мы будем доставать дорогую нашему исследовательскому сердцу информацию выглядит следуюшим образом:

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/doge_main.png" height="600" width="600"> 

Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [ ]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить статистику просмотров, комментариев, а также числа загруженных видео о фото, связанных с нашим мемов. Всё это добро хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [ ]:
views = soup.find('dd', attrs={'class':'views'})
views

In [ ]:
views = views.find('a').text
views

In [ ]:
views = int(views.replace(',', ''))
views

Снова апихнём всё это в небольшую функцию.

In [ ]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
        stats: string
            views/videos/photos/comments
            
    """

    obj = soup.find('dd', attrs={'class':stats})
    obj = obj.find('a').text
    obj = int(obj.replace(',', ''))
    
    return obj

Всё готово! 

In [ ]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Еще из инетерсного —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем вытащить - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, полезем в кишки html и откопаем там кусок, ответственный за эту надпись:

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/no_shit.jpg" height="300" width="300"> 

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/html_time_ago.png" height="600" width="600"> 

Ага! Вот и подробности по дате добавления, с точностью до минуты. Элементарно

In [ ]:
date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']
date

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится прописывать исключения. Вроде бы, хранилище мемов хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, мы будем использовать на каждой итерации конструкцию `try - except`, которая часто выручает при собирательстве. 

Если кто не знает, `try - except` — это очень крутая конструкция, которая позволяет прописывать исключения. Посмотрим на то как она работает: 

```python
try:
    something
except Name_of_some_error:
    do_something_else
```

> Конструкция позволяет в случае, если произошла ошибка типа Name_of_some_error не заканчивать программу со словами «Все пропало! Ошибка!», а тут же передать управление блоку do_something_else, который что-нибудь сделает. 

Например, в ситуации ниже, когда функция сталкивается с отрицательными числами, она не пишет нам, что логарифма от отрицательного числа не существует, она молча записывает в вектор `b` указанное нами число. 

In [ ]:
from math import log
a = [1,2,3,4,-5,-7,9,10,0,12,2]
b = [ ]
for atem in a:
    try:
        b.append(log(atem))
    except:
        b.append('NA')
print(b)

Про исключения можно почитать [на просорах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В данном примере мы наложили очень сильное исключение, которое работает при возникновении абсолютно любой неадекватной ситуации в коде. Это исключение при необходимость можно ослабить, заменив на другое. 

Например, код ниже пытается найти статус мема и успешно находит. 

In [ ]:
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")
meme_status

Дальше нужно вытащить из тэгов текст, а после обрубить все лишние пробелы. 

In [ ]:
meme_status.text.strip()

Однако, если неожиданно выяснится, что у мема нет статуса, метод `find` вернёт пустоту. Метод `text`, в свою очередь, не сможет найти в тэгах текст и выдаст ошибку. Чтобы обезопасить себя от таких пустот, можно прописать исключение.

In [ ]:
# Делай раз! Находим статус мема!
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")

# Делай два! Пытаемся вытащить его 
try:
    print(meme_status.text.strip()) 
# Ежели возникает ошибка, статус не найден, выдаём пустоту.
except:
    print("")

Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с исключениями в виде одной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [ ]:
meme_status = "" if not meme_status else meme_status.text.strip()
print(meme_status)

По аналогии можно вытащить всю остальную информацию со страницы.

In [ ]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом, 
        годом и местом происхождения и тэгами
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
    
    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()
    
    # достаём все данные справа от картинки 
    properties = soup.find('aside', attrs={'class':'left'})
    
    # статус идет первым - можно не уточнять класс
    meme_status = properties.find("dd")
    # oneliner, заменяющий try-except: если тэга нет в properties, вернётся объект NoneType,
    # у которого аттрибут text отсутствует, и в этом случае он заменится на пустую строку
    meme_status = "" if not meme_status else meme_status.text.strip()
    
    # тип мема - обладает уникальным классом
    meme_type = properties.find('a', attrs={'class':'entry-type-link'})
    meme_type = "" if not meme_type else meme_type.text 
    
    # год происхождения первоисточника можно найти после заголовка Year, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin_year = properties.find(text='\nYear\n')
    meme_origin_year = "" if not meme_origin_year else meme_origin_year.parent.find_next()
    meme_origin_year = meme_origin_year.text.strip()
    
    # сам первоисточник
    meme_origin_place = properties.find('dd', attrs={'class':'entry_origin_link'})
    meme_origin_place = "" if not meme_origin_place else meme_origin_place.text.strip()
    
    # тэги, связанные с мемом
    meme_tags = properties.find('dl', attrs={'id':'entry_tags'}).find('dd')
    meme_tags = "" if not meme_tags else meme_tags.text.strip()
    
    return meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags

In [ ]:
getProperties(soup)

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание. 

In [ ]:
def getText(soup):
    """
        Возвращает текстовые описания мема
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
    """
    
    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})
    
    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text
    
    # раздел origin можно найти после заголовка Origin или History, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text
    
    # весь остальной текст (если он есть) можно запихнуть в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""
        
    return meme_about, meme_origin, other_text

In [ ]:
meme_about, meme_origin, other_text = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(meme_about, meme_origin, other_text[:200]))

Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [ ]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        meme_page: string
            ссылка на страницу с мемом
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']

    # имя, статус, и т.д.
    meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags =\
    getProperties(soup=soup)

    # текстовые поля
    meme_about, meme_origin, other_text = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"name":meme_name, "status":meme_status, 
                "type":meme_type, "origin_year":meme_origin_year, 
                "origin_place":meme_origin_place,
                "date_added":date, "views":views, 
                "videos":videos, "photos":photos, "comments":comments, "tags":meme_tags,
                "about":meme_about, "origin":meme_origin, "other_text":other_text}

    return data_row

In [ ]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [ ]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

In [ ]:
final_df = final_df.append(data_row, ignore_index=True)

In [ ]:
final_df

Первый мем оказался в наших рукак. Именно в этот момент мы понимаем, что покой это ложь — есть только страсть. В текущий момент — это страсть к сбору мемов. Еще раз убедимся что всё работает — пройдемся по списку из ссылок мемом, полученных ранее в перменной `meme_links`.

In [ ]:
for meme_link in meme_links:
    data_row = getMemeData(meme_link)
    final_df = final_df.append(data_row, ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.shape

Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2. Когда работающий код больше не работает

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем $1075$ страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает. 


**Для Димы: какого-то хера остаются проблемы с некоторыми мемами. Я так и не смог их пофиксить (((**

In [ ]:
# Немного пафосных циклов. При желании пакет можно отключить и 
# удалить команду tqdm_notebook из всех циклов
from tqdm import tqdm_notebook

final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])


for page_number in tqdm_notebook(range(1075)):
    meme_links = getPageLinks(page_number)               # собрали хрефы с текущей страницы
    for meme_link in meme_links:
        try:
            data_row = getMemeData(meme_link)            # пытаемся собрать по мему немного даты
            final_df = final_df.append(data_row, ignore_index=True)  # и закидываем её в таблицу
        except:
            # Иначе, ахтунг! 
            print('AHTUNG! Meme not found:', meme_link)

In [ ]:
final_df.shape

Сон был прекрасным! Солце только-только взошло из-за горизонта, мы уже бежим за компьютер смотреть мемы и видим, что огромное число мемов не скачалось.

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/work_or_not.jpg" height="500" width="500">

Конечно же, вполне естественной реакцией будет нажать на первую же ссылку, перейти в мемохранилище и увидеть, что нас забанили.

<img align="center" src="https://raw.githubusercontent.com/DmitrySerg/memology/master/pictures/memes_ban.png" height="600" width="600"> 

Все наши реквесты остались без респонсов. 

In [ ]:
getMemeData('http://knowyourmeme.com/memes/doge')

# 3. Тор — сын Одина

Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Иногда серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его. К сожалению, не только люди обладают вздорным характером... 

Приходится маскироваться. Для такой маскировки довольно удобно использовать возможности Tor. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам: 

* ссылки 

Мы же ограничимся только функциональной частью, а именно без углубления в детали опишем шаги, которые нужно предпринять для того, чтобы использовать возможности Tor для обхода блокировки. Для начала полюбуемся на свой ip-адрес. Для этого сделаем get-запрос к сайту, который был создан злобными школьниками (если кто не понял, он знает наш ip).

In [ ]:
requests.get('http://icanhazip.com').content

Заменить свой ip через Tor можно двумя путями. Сложным и более простым, если со сложным у нас не очень сложилось. Сначала попробуем пойти более простой, браузерной дорогой.

Скачаем браузерный [tor,](https://www.torproject.org/download/download) чтобы лёгкий путь был совсем прост. Для сложного пути в довесок к браузеру поставим tor через консоль. На линуксе нам поможет команда `apt-get install tor`, на маке сделаем это [в рамках brew](https://www.torproject.org/docs/tor-doc-osx.html.en), `brew install tor`. На windows нам поможет переустановка операционной системы. 

## 3.1 Путь первый 

Теперь открываем браузер и оставляем его открытым.  Менять ip нам поможет библиотека `PySocks`. Конечно же её нужно установить, скопировав в терминал `pip3 install PySocks`.  

**Диме:  бла бла бла как это работает (я хз как это описать без слова "магия"**

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

Посмотрим на свой новый ip-aдрес. 

In [ ]:
requests.get('http://icanhazip.com', headers={'User-Agent': UserAgent().chrome}).content

При сопутствующем желании, [можно выяснить](http://ru.smart-ip.net/geoip) из какой страны в данный момент мы сидим в интернете.

Ого... Германия.

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/ip_adress.png" height="250" width="250"> 

Попробуем обратиться к мемохранилищу с нового ip-адреса.

In [ ]:
getMemeData('http://knowyourmeme.com/memes/doge')

Бан снят. Стражники мемов ничего не заподозрили и пустили нас в сокровищницу. Чашу нашего респонса снова переполняет контент. Через силу мы обрели мощь. 

При желании, можно выяснить одну занимательную вещь: при базовых настройках, Тор-браузер меняет ip каждые 10 минут. Но что делать, если сервер банит нас быстрее?  Всё просто, в папке, куда был установлен Tor найдём файлик с настройками под названием torrc и отредактируем его. Добавим строки: 

```
CircuitBuildTimeout 10
LearnCircuitBuildTimeout 0
MaxCircuitDirtiness 10
```

Минимально возможные период для обновления ip составляет 10 секунд. Установим туда эту цифру и попробуем поиграться. 

In [ ]:
for i in range(10):
    print(requests.get('http://icanhazip.com').content)
    time.sleep(5)

Действительно, смена ip происходит раз в 10 секунд. Для наших целей по скачке мемов было достаточно и базовых настроек. Бан наступал примерно через 20 минут после начала работы кода. 

1. Открываем браузер;
2. Запускаем кусок кода с подгрузкой библиотек;
3. Запускаем цикл по мемам 
4. .....
5. Profit

In [ ]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])


for page_number in tqdm_notebook(range(1075)):
    meme_links = getPageLinks(page_number)               # собрали хрефы с текущей страницы
    for meme_link in meme_links:
        try:
            data_row = getMemeData(meme_link)            # пытаемся собрать по мему немного даты
            final_df = final_df.append(data_row, ignore_index=True)  # и закидываем её в таблицу
        except:
            # Иначе, ахтунг! 
            print('AHTUNG! Meme not found:', meme_link)
            
final_df.shape

Все мемы в наших руках. Можно приступать к варки фичей и моделированию. Через мощь мы познали победу. Остался только один вопрос: **Что, если мы хотим менять ip каждый реквест?**

## 3.2 Пуь второй 

Второй путь помогает извращаться со сменой ip как угодно. Зайдём на [Github каких-то ребят](https://github.com/alex-miller-0/Tor_Crawler) и скачаем себе их скрипт под названием `TorCrawler.py`. Все недостающие библиотеки, используемые в этом скрипте, придётся доставить. Поставим парням на репозиторий за их код звёздочку. Закинем этот скрипт либо в папку со своими библиотеками либо в папку к этому блокноту. Обратите внимание, что скрипт работает только с третьим питоном. 

Перед использованием библиотечки, нужно подкрутить настройки в torrc файлике. На маке он будет лежать в папке `/usr/local/etc/tor/`, на линуксе в папке `/etc/tor/`. Проследуем по инструкции авторов скрипта. 

**Для Димы: мб как-то переписать более научно?..** 

1. Генерируем пароль `tor --hash-password mypassword` 
2. Открываем torrc файл в редакторе вроде vim или [atom](https://atom.io/) 
3. Сохраняем пароль в наш torrc файл в строку, которая начинается с HashedControlPassword
4. Раскоментируем строку, начинающуюся с HashedControlPassword
5. Раскоментируем (если она закоментирована) строку ControlPort 9051
6. Сохраним изменения. 

Запускаем tor в терминале. Линукс: `service tor start`, мак: `tor`.

Теперь мы готовы парсить. Подгружаем скрипт в наш блокнот. 

In [ ]:
from TorCrawler import TorCrawler

In [ ]:
# Создаём свой краулер, в опциях вводим пароль 
crawler = TorCrawler(ctrl_pass='mypassword')    
crawler.ip # Смотрим на свой ip 

Мы можем сделать get-запрос по аналогии с тем как делали раньше. 

In [ ]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = crawler.get(meme_page, headers={'User-Agent': UserAgent().chrome})

Получаем ответ сразу же в формате bs4.

In [ ]:
type(response)

Находим внутри что-нибудь нужное. 

In [ ]:
views = response.find('dd', attrs={'class':'views'})
views

По дефолту краулер меняет ip каждые $n$ запросов. За это отвечает параметр `n_requests`. По дэфолту он равен 25. Однако при создании нового краулера, его можно настроить по собственному желанию. 

In [ ]:
crawler.n_requests

Более того, ip можно при желании поменять вручную. 

In [ ]:
crawler.rotate()

In [ ]:
crawler.ip

** Для Димы: Каких ещё приколов сюда добавить?** 

В принципе, весь код выше можно перевести на рельсы этого небольшого скрипта, заменив все реквесты на торовские. 

Победа порвала наши оковы. Великая Сила освободла нас.

## Почиташки 

* [Годная книга](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) про парсинг на английском языке. 
* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов. 
* Димин репозиторий с исследованием мемов и другими ништяками.
* [Филин репозиторий](https://fulyankin.github.io/Parsers/) с ещё парой хорошо расписаных блокнотов с парсерами. Репозиторий изначально делался как страничка факультатива для студентов. 
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.

**Диме : может заменить кодекс ситов на какие-нибудь скандинавские приколы?** 
